In [1]:
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.decomposition import PCA
import cProfile
from sklearn.preprocessing import StandardScaler
from scipy.sparse import csr_matrix
from sklearn.utils import resample
from sklearn.impute import KNNImputer
from tqdm import tqdm
import concurrent.futures

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.neighbors import KNeighborsClassifier

from concurrent.futures import ProcessPoolExecutor, as_completed

from model_functions import logistic_test, svm_test, nb_test, rf_test, gb_test, ensemble_test, stacking_test, run_model

import warnings

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import joblib

In [ ]:
'''def main():
    # Read the datasets
    ingredients_df = pd.read_csv('data/ingredients_df_balanced.csv')
    flavour_df = pd.read_csv('data/flavour_df_balanced.csv')
    region_profile_df = pd.read_csv('data/region_profile_df_balanced.csv')
    merged_df = pd.read_csv('data/merged_df.csv')

    # Separate features and target
    X = merged_df.drop(columns=['id', 'region'])
    y = merged_df['region']

    # Apply PCA for dimensionality reduction
    pca = PCA(n_components=0.95)  # retain 95% of variance
    X = pca.fit_transform(X)

    # Split the data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Scale the features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Define models to test
    models = [logistic_test, svm_test, nb_test, rf_test, gb_test, knn_test, ensemble_test, stacking_test]

    # Run models in parallel with progress bar
    results = []
    with ProcessPoolExecutor() as executor:
        futures = {executor.submit(run_model, model, X_train_scaled, X_test_scaled, y_train, y_test): model for model in models}
        for future in tqdm(as_completed(futures), total=len(futures), desc="Running models"):
            results.append(future.result())

    # Display results
    for model_name, accuracy in results:
        print(f"{model_name} - Accuracy: {accuracy}")

    # Plot confusion matrix with Ensemble model
    ensemble = StackingClassifier(estimators=[
        ('lr', LogisticRegression(max_iter=1000)),
        ('svc', SVC(kernel='linear', probability=True)),
        ('rf', RandomForestClassifier()),
        ('knn', KNeighborsClassifier())
    ], final_estimator=GradientBoostingClassifier())
    ensemble.fit(X_train_scaled, y_train)
    y_pred = ensemble.predict(X_test_scaled)
    cm = confusion_matrix(y_test, y_pred)
    cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    plt.figure(figsize=(10, 10))
    plt.imshow(cm_normalized, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title('Confusion Matrix based on Ensemble Model')
    plt.colorbar()
    tick_marks = np.arange(len(np.unique(y)))
    plt.xticks(tick_marks, np.unique(y), rotation=45)
    plt.yticks(tick_marks, np.unique(y))
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()
    plt.show()
'''

In [ ]:
def main():
    # Read the datasets
    merged_df = pd.read_csv('data/merged_df_imputed.csv')

    # Separate features and target
    X = merged_df.drop(columns=['id', 'region'])
    y = merged_df['region']

    # Split the data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Scale the features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Define models to test
    models = [logistic_test, svm_test, nb_test, rf_test, gb_test, ensemble_test, stacking_test]

    results = []
    with ProcessPoolExecutor() as executor:
        futures = {executor.submit(run_model, model, X_train_scaled, X_test_scaled, y_train, y_test): model for model in models}
        for future in tqdm(as_completed(futures), total=len(futures), desc="Running models"):
            results.append(future.result())

    # Display results
    for model_name, accuracy in results:
        print(f"{model_name} - Accuracy: {accuracy}")

    # Plot confusion matrix with Ensemble model
    ensemble = StackingClassifier(estimators=[
        ('lr', LogisticRegression(max_iter=1000)),
        ('svc', SVC(kernel='linear', probability=True)),
        ('rf', RandomForestClassifier())
    ], final_estimator=GradientBoostingClassifier())
    ensemble.fit(X_train_scaled, y_train)
    y_pred = ensemble.predict(X_test_scaled)
    cm = confusion_matrix(y_test, y_pred)
    cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    plt.figure(figsize=(10, 10))
    plt.imshow(cm_normalized, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title('Confusion Matrix based on Ensemble Model')
    plt.colorbar()
    tick_marks = np.arange(len(np.unique(y)))
    plt.xticks(tick_marks, np.unique(y), rotation=45)
    plt.yticks(tick_marks, np.unique(y))
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()
    plt.show()


In [ ]:
if __name__ == '__main__':
    main()

In [5]:
    # Read the datasets
merged_df = pd.read_csv('data/merged_df_imputed.csv')

# Separate features and target
X = merged_df.drop(columns=['id', 'region'])
y = merged_df['region']

In [6]:
 # Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [7]:
X_train

,compound_count,ingredient_count,almond,angelica,anise,anise_seed,apple,apple_brandy,apricot,armagnac,...,1097,1098,1099,1100,1101,1102,1103,1104,1105,1106
3593,245,6,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0
794,430,13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,1.0,3.0,0.0,4.0,0.0,1.0
6568,229,8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.0,0.0,3.0,0.0,0.0
1533,272,8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0
1558,215,9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3772,309,9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0
5191,228,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0
5226,202,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0
5390,282,14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,3.0,0.0,1.0,0.0,0.0


In [8]:
X_train.to_csv('data/X_train.csv', index=False)

In [4]:
joblib.dump(scaler, 'scaler.pkl')

['scaler.pkl']

In [ ]:
def ensemble_test(X_train, X_test, y_train, y_test):
    estimators = [
        ('lr', LogisticRegression(max_iter=1000)),
        ('svc', SVC(kernel='linear', probability=True)),
        ('rf', RandomForestClassifier())
    ]
    ensemble = StackingClassifier(estimators=estimators, final_estimator=GradientBoostingClassifier())
    ensemble.fit(X_train, y_train)
    y_pred = ensemble.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    return 'Ensemble', acc

In [ ]:
ensemble_test(X_train_scaled, X_test_scaled, y_train, y_test)

In [ ]:
def train_ensemble_model(X_train, y_train):
    estimators = [
        ('lr', LogisticRegression(max_iter=1000)),
        ('svc', SVC(kernel='linear', probability=True)),
        ('rf', RandomForestClassifier())
    ]
    ensemble = StackingClassifier(estimators=estimators, final_estimator=GradientBoostingClassifier())
    ensemble.fit(X_train, y_train)
    return ensemble

# Train the model
ensemble = train_ensemble_model(X_train_scaled, y_train)

In [ ]:
joblib.dump(ensemble, 'pretrained_ensemble_model.pkl')

In [ ]:
from sklearn.model_selection import cross_val_score, StratifiedKFold
def cross_validate_ensemble(X, y):
    estimators = [
        ('lr', LogisticRegression(max_iter=1000)),
        ('svc', SVC(kernel='linear', probability=True)),
        ('rf', RandomForestClassifier())
    ]

    ensemble = StackingClassifier(estimators=estimators, final_estimator=GradientBoostingClassifier())

    # Define the cross-validation strategy
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    # Perform cross-validation
    cv_scores = cross_val_score(ensemble, X, y, cv=cv, scoring='accuracy', n_jobs=-1)

    # Fit the model on the entire dataset
    ensemble.fit(X, y)

    return ensemble, cv_scores

cross_validate_ensemble(X_train_scaled, y_train)

In [ ]:
merged_df = pd.read_csv('data/merged_df_imputed.csv')

In [ ]:
ingredients_df = pd.read_csv('data/ingredients_df.csv')

In [ ]:
def add_id_column(df, column_name='id', start=1):
    """
    Add an ID column to the DataFrame.
    
    Parameters:
    - df: DataFrame to which the ID column will be added.
    - column_name: Name of the ID column (default is 'id').
    - start: Starting value of the ID (default is 1).
    
    Returns:
    - DataFrame with the added ID column.
    """
    df.insert(0, column_name, range(start, start + len(df)))
    return df

# Example usage:
# Assuming ingredients_df is your DataFrame
ingredients_df = add_id_column(ingredients_df)


In [ ]:
# Filter North American cuisine
north_american_df = ingredients_df[ingredients_df['region'] == 'north_american']

# Randomly sample 5000 observations
north_american_sampled_df = north_american_df.sample(n=3000, random_state=42)

# Combine the sampled North American data with the rest of the DataFrame
other_df = ingredients_df[ingredients_df['region'] != 'north_american']
ingredients_df = pd.concat([other_df, north_american_sampled_df], ignore_index=True)

In [ ]:
ingredients_df

In [ ]:
# Assuming ingredients_df is already defined and loaded

# Find the size of the median class
median_class_size = ingredients_df['region'].value_counts().median()

# Separate each region into individual DataFrames
dfs = [df for _, df in ingredients_df.groupby('region')]

# Initialize KNN Imputer
knn_imputer = KNNImputer(n_neighbors=5)

# Initialize OneHotEncoder for categorical data
categorical_cols = ingredients_df.select_dtypes(include=['object']).columns
numeric_cols = ingredients_df.select_dtypes(exclude=['object']).columns

ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

# Resample or impute each region to the median class size
balanced_dfs = []
for df in dfs:
    if len(df) < median_class_size:
        # Resample with replacement to median size
        resampled_df = resample(df, replace=True, n_samples=int(median_class_size), random_state=42)

        # Encode categorical columns
        encoded_categorical = ohe.fit_transform(resampled_df[categorical_cols])
        encoded_categorical_df = pd.DataFrame(encoded_categorical, columns=ohe.get_feature_names_out(categorical_cols))

        # Combine numeric and encoded categorical columns
        combined_df = pd.concat([resampled_df[numeric_cols].reset_index(drop=True), encoded_categorical_df.reset_index(drop=True)], axis=1)

        # Apply KNN Imputer
        imputed_combined = knn_imputer.fit_transform(combined_df)
        imputed_df = pd.DataFrame(imputed_combined, columns=combined_df.columns)

        # Separate back the numeric and categorical columns
        imputed_numeric = imputed_df[numeric_cols]
        imputed_categorical = pd.DataFrame(ohe.inverse_transform(imputed_df[encoded_categorical_df.columns]), columns=categorical_cols)

        # Combine into final DataFrame
        final_df = pd.concat([imputed_numeric, imputed_categorical], axis=1)
        balanced_dfs.append(final_df)
    else:
        # Resample without replacement to median size
        resampled_df = resample(df, replace=False, n_samples=int(median_class_size), random_state=42)
        balanced_dfs.append(resampled_df)

# Combine the DataFrames
balanced_df = pd.concat(balanced_dfs, ignore_index=True)

# Update ingredients_df with the balanced DataFrame
ingredients_df = balanced_df.copy()

# Verify the new counts
balanced_region_counts = ingredients_df['region'].value_counts()
balanced_country_counts = ingredients_df['country'].value_counts()

# Plot the data for regions
plt.figure(figsize=(10, 8))
sns.barplot(x=balanced_region_counts.values, y=balanced_region_counts.index, palette='viridis')
plt.title('Number of Entries per Region (Balanced)')
plt.xlabel('Number of Entries')
plt.ylabel('Region')
plt.tight_layout()
plt.show()

# Plot the data for countries
plt.figure(figsize=(10, 8))
sns.barplot(x=balanced_country_counts.values, y=balanced_country_counts.index, palette='viridis')
plt.title('Number of Entries per Country (Balanced)')
plt.xlabel('Number of Entries')
plt.ylabel('Country')
plt.tight_layout()
plt.show()


In [ ]:
def flavor_profile(ingredients_df, ingr_df, comp_df, ingr_comp_df):
    ingredients_df.reset_index(drop=True, inplace=True)
    ingredients_df['id'] = ingredients_df.index

    function_df = ingredients_df.copy()
    undesired_columns = ['region', 'id']  # Keep 'id' out of calculations
    function_df.drop(undesired_columns, axis=1, inplace=True)
    sorted_ingredients = function_df.columns

    ingr_total = ingr_comp_df.merge(ingr_df, how='right', on='ingredient_id')
    ingr_total = ingr_total.merge(comp_df, how='right', on='compound_id')

    ingr_pivot = pd.crosstab(ingr_total['ingredient_name'], ingr_total['compound_id'])
    ingr_flavor = ingr_pivot.reindex(sorted_ingredients).fillna(0)

    df_flavor = pd.DataFrame(np.dot(function_df.values, ingr_flavor.values), index=ingredients_df.index)
    df_flavor['region'] = ingredients_df['region']
    df_flavor['id'] = ingredients_df['id']  # Add 'id' column

    # Reorder columns to have 'id' first
    cols = ['id'] + [col for col in df_flavor.columns if col != 'id']
    df_flavor = df_flavor[cols]

    # Reorder columns to have 'region' and 'country' first
    column_order = ['region'] + [col for col in df_flavor.columns if col not in ['region']]
    df_flavor = df_flavor[column_order]

    return df_flavor

def add_flavour_profile(ingredients_df, flavour_df):
    # Ensure 'id' is included as a unique identifier
    ingredients_df['id'] = range(len(ingredients_df))
    flavour_df['id'] = ingredients_df['id']

    # Ensure that 'flavour_df' and 'ingredients_df' have the same index
    flavour_df.index = ingredients_df.index
    ingredients_df.index = ingredients_df.index

    # Create a new DataFrame
    region_recipe_df = pd.DataFrame()
    region_recipe_df['id'] = ingredients_df['id']
    region_recipe_df['region'] = ingredients_df['region']

    # Calculate the total count of compounds per recipe from flavour_df
    region_recipe_df['compound_count'] = np.count_nonzero(flavour_df.values, axis=1)

    # Calculate the total count of ingredients per recipe from ingredients_df
    ingredient_columns = [col for col in ingredients_df.columns if col not in ['id', 'region']]
    region_recipe_df['ingredient_count'] = np.count_nonzero(ingredients_df[ingredient_columns].values, axis=1)

    return region_recipe_df



In [ ]:
ingredients_df

In [ ]:
ingredients_df.to_csv('data/ingredients_df_imputed.csv', index=False)

In [ ]:
flavour_df.to_csv('data/flavour_df_imputed.csv', index=False)
region_profile_df.to_csv('data/region_profile_df_imputed.csv', index=False)

In [ ]:
# Load the data
comp_path = 'data/flavor_network_data/ingr_comp/comp_info.tsv'
comp_tsv = pd.read_csv(comp_path, delimiter='\t')

comp_df = pd.DataFrame(data = comp_tsv)
comp_columns = ['compound_id', 'compound_name', 'CAS_number']
comp_df.columns = comp_columns

ingr_path = 'data/flavor_network_data/ingr_comp/ingr_info.tsv'
ingr_tsv = pd.read_csv(ingr_path, delimiter='\t')

ingr_df = pd.DataFrame(data = ingr_tsv)
ingr_columns = ['ingredient_id', 'ingredient_name', 'ingredient_category']
ingr_df.columns = ingr_columns

ingr_comp_pathh = 'data/flavor_network_data/ingr_comp/ingr_comp.tsv'
ingr_comp_tsv = pd.read_csv(ingr_comp_pathh, delimiter='\t')

ingr_comp_df = pd.DataFrame(data = ingr_comp_tsv)
ingr_comp_df.rename(columns={
    '# ingredient id': 'ingredient_id',
    'compound id': 'compound_id'
}, inplace=True)

In [ ]:
ingredients_df = pd.read_csv('data/ingredients_df.csv')

In [ ]:
ingredients_df = pd.read_csv('data/ingredients_df.csv')
flavour_df = pd.read_csv('data/flavour_df_imputed.csv')
region_profile_df = pd.read_csv('data/region_profile_df_imputed.csv')

In [ ]:
ingredients_df

In [ ]:
ingredients_df = ingredients_df.drop(columns=['country'])

In [ ]:
ingredients_df

In [ ]:
flavour_df = flavor_profile(ingredients_df, ingr_df, comp_df, ingr_comp_df)

In [ ]:
region_profile_df = add_flavour_profile(ingredients_df, flavour_df)

In [ ]:
# Drop the 'country' column from ingredients_df
flavour_df = flavour_df.drop(columns=['region'])

# Drop the 'country' column from flavour_df
ingredients_df = ingredients_df.drop(columns=['region'])

In [ ]:
ingredients_df

In [ ]:
# Reset the index to ensure it's unique and sequential, if needed
ingredients_df.reset_index(drop=True, inplace=True)

# Add an ID column based on the new index
ingredients_df['id'] = ingredients_df.index

In [ ]:
# Ensure 'id' is the first column
cols = list(ingredients_df.columns)
cols.insert(0, cols.pop(cols.index('id')))

# Ensure 'region' is the second column
cols.insert(1, cols.pop(cols.index('region')))

# Reorder the DataFrame
ingredients_df = ingredients_df[cols]

In [ ]:
ingredients_df

In [ ]:
flavour_df

In [ ]:
 # Merge the dataframes on 'id'
merged_df = region_profile_df.merge(ingredients_df, on='id').merge(flavour_df, on='id')

In [ ]:
merged_df

In [ ]:
merged_df.to_csv('data/merged_df_imputed.csv', index=False)

In [ ]:
pd.Series(y_pred)

In [ ]:
cm = confusion_matrix(y_test, y_pred)
np.set_printoptions(precision=2)
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
plt.figure(figsize=(10, 10))
cuisine_labels = sorted(y['region'].unique())
plot_confusion_matrix_1(cm_normalized, cuisine_labels, title='Confusion Matrix based on ingredients')
plt.show()